
# Demo: اتصال به داده‌ی آنچین (Glassnode-like) + شبیه‌سازی در نبود اینترنت

> این نوت‌بوک نشان می‌دهد چگونه به یک API آنچین متصل شوید و اگر اینترنت/کلید API در دسترس نبود، با داده‌ی ساختگی روندها را تمرین کنید.


In [ ]:
# تنظیمات اولیه
import os, math, json, random, datetime as dt
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

ASSET = "BTC"  # یا 'ETH', ...
START = dt.date.today() - dt.timedelta(days=365)
DATES = pd.date_range(START, periods=365, freq="D")

# حالت شبیه‌سازی (بدون اینترنت)
np.random.seed(42)
price = pd.Series(20000 + np.cumsum(np.random.randn(len(DATES))*50), index=DATES)
tx_volume_usd = pd.Series(1e9 + np.cumsum(np.random.randn(len(DATES))*1e7), index=DATES).abs()
active_addresses = pd.Series(700000 + np.random.randn(len(DATES))*20000, index=DATES).abs()
supply = 19_600_000  # تقریب
market_cap = price * supply
nvt = (market_cap / tx_volume_usd).rolling(14).mean()

# SOPR شبیه‌سازی‌شده (نزدیک 1 با نوسان اندک)
sopr = pd.Series(1 + np.random.randn(len(DATES))*0.02, index=DATES)

# Netflow صرافی‌ها (کوین/روز)
netflow = pd.Series(np.random.randn(len(DATES))*5000, index=DATES)

df = pd.DataFrame({
    "price": price,
    "tx_volume_usd": tx_volume_usd,
    "active_addresses": active_addresses,
    "market_cap": market_cap,
    "nvt": nvt,
    "sopr": sopr,
    "exchange_netflow": netflow
})
df.head()



## ترسیم سری‌های اصلی
> توجه: برای رسم نمودارها از matplotlib استفاده شده و استایل/رنگ خاصی تعیین نشده است.


In [ ]:
# قیمت
plt.figure()
df["price"].plot(title="Price (simulated)")
plt.show()

# NVT
plt.figure()
df["nvt"].plot(title="NVT (14D MA, simulated)")
plt.show()

# SOPR
plt.figure()
df["sopr"].plot(title="SOPR (simulated)")
plt.axhline(1.0)
plt.show()

# Exchange Netflow
plt.figure()
df["exchange_netflow"].plot(title="Exchange Netflow (simulated)")
plt.axhline(0.0)
plt.show()



## اتصال واقعی به API (قالب کد — اجرا فقط با اینترنت/کلید)
> این بخش نمونه‌کد است و در این محیط اجرا نمی‌شود.


In [ ]:
# قالب فراخوانی API (نمونه نمایشی؛ آدرس‌ها را با مستندات سرویس واقعی تطبیق دهید)
# import requests
# API_KEY = os.getenv("GLASSNODE_API_KEY")
# headers = {"X-Api-Key": API_KEY}
# url = "https://api.glassnode.com/v1/metrics/addresses/active_count"
# params = {"a": ASSET, "i": "24h", "s": int(pd.Timestamp(START).timestamp())}
# r = requests.get(url, headers=headers, params=params, timeout=30)
# data = r.json()
# df_api = pd.DataFrame(data)
# df_api["t"] = pd.to_datetime(df_api["t"], unit="s")
# df_api = df_api.set_index("t").rename(columns={"v": "active_addresses"})
# df_api.head()



## سیگنال‌های ساده
- **SOPR < 1** به‌صورت پایدار → فشار فروش؛ احتمال کف‌سازی.
- **NVT بالا** نسبت به تاریخچه → ارزش‌گذاری کش‌آمده/خطر اصلاح.
- **Exchange Netflow مثبت بزرگ** → افزایش احتمالی عرضه‌ی فروش.


In [ ]:
signals = {}
signals["sopr_below_1_days"] = int((df["sopr"] < 1.0).tail(14).sum())
signals["nvt_percentile_90"] = float(df["nvt"].dropna().quantile(0.9))
signals["nvt_is_high"] = bool(df["nvt"].iloc[-1] > signals["nvt_percentile_90"])
signals["netflow_extreme"] = bool(abs(df["exchange_netflow"].iloc[-1]) > df["exchange_netflow"].std()*2)

signals



## نکات عملیاتی
- برچسب صرافی‌ها را از چند منبع متقاطع کنید.
- نرمال‌سازی مقادیر با موجودی صرافی/مارکت‌کپ.
- ثبت عدم قطعیت در گزارش‌ها (±).
